In [14]:
import warnings
warnings.filterwarnings('ignore')

# 교차 검증과 그리드 서치
- 머신러닝을 사용할때 모델의 정확도를 측정하기 위해 반드시 사용해야 하는 방법.
- 딥러닝시에는 데이터의 크기가 크므로 이 방법은 사용할 필요가 없다.


In [15]:
import pandas as pd
wine = pd.read_csv("../Data/wine.csv")
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [16]:
# Feature, Target 
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

# 검증 세트 추가 


In [17]:
# 전체 세트 중 훈련세트와 테스트 세트를 8 : 2 의 기준으로 분리한다.
from sklearn.model_selection import train_test_split
train_input,test_input, train_target, test_target = train_test_split(
    data, target, test_size= 0.2, random_state=42
)

In [18]:
# 훈련세트 중 훈련세트와 검증세트를 8:2의 기준으로 분리한다
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input,train_target,test_size=0.2, random_state=42
)

In [19]:
# 훈련세트, 검증세트, 테스트 세트의 크기 구하기
print("훈련세트 :", sub_input.shape)
print("검증세트 :", val_input.shape)
print("테스트세트 :", test_input.shape)

훈련세트 : (4157, 3)
검증세트 : (1040, 3)
테스트세트 : (1300, 3)


In [20]:
# 훈련세트와 검증세트를 결정트리로 모델 만들기
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)

print("Train score :",dt.score(sub_input,sub_target))
print("valid score :",dt.score(val_input,val_target))

Train score : 0.9971133028626413
valid score : 0.864423076923077


---
# 교차검증

In [21]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.0059042 , 0.00554585, 0.00544596, 0.0057478 , 0.00498104]),
 'score_time': array([0.0006609 , 0.00067616, 0.00034404, 0.00047898, 0.00031686]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [22]:
import numpy as np
np.mean(scores['test_score'])

0.855300214703487

---
# KFold : 분할기를 사용한 교차 검증

In [27]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=2) # 기본 n_splits = 5
scores = cross_validate(dt, train_input, train_target, cv=splitter)
scores

{'fit_time': array([0.00399208, 0.00377297]),
 'score_time': array([0.00085783, 0.00075817]),
 'test_score': array([0.85879184, 0.85450346])}

In [28]:
np.mean(scores['test_score'])

0.856647653609889

In [30]:
# KFold의 Fold를 10개로 나누어서 교차검증
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8574181117533719

In [32]:
# KFold의 Fold를 10개로 나누어서 교차검증
splitter = StratifiedKFold(n_splits=10)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8616407292129834

In [ ]:
# shuffle 해줘야지 예측시에 값을 따라가지않음 -> 그렇게 만든게 랜덤포레스트 